In [1]:
pip install numpy


The following command must be run outside of the IPython shell:

    $ pip install numpy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [ ]:
pip install abjad

In [63]:
import ipywidgets as widgets
from IPython.display import clear_output
from random import choice, randrange
from numpy.random import choice as npchoice
import abjad

In [41]:

pitches = widgets.IntRangeSlider(
    value=[0, 11],
    min=0,
    max=11, # make this larger to include more than one octave
    description='Pitch Range:', # chromatic pitches
)
display(pitches)

pitch_range = max(pitches.value) - min(pitches.value) + 1

counts = [0] * 12 # counts since that index of pitches was chosen
pcprob = [1] * 12 # pitch class probability
weights = [1] * 12 # how much to weigh each element's probability

offset = 0
# Number of pitches in sequence
length = widgets.IntSlider(
    value=pitch_range,
    min=pitch_range,
    max=500,
    step=1,
    description='Length (# notes):',
    style={'description_width': 'initial', 'width': '800px'},
    readout=True,
)
display(length)
    


IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=12, description='Length (# notes):', max=500, min=12, style=SliderStyle(description_width='ini…

In [61]:
pitch_range = max(pitches.value) - min(pitches.value) + 1
pitch_range

12

In [64]:
def make_random_pitch_sequence(pitches, offset, length):
    sequence = []
    for note in range(length.value):
        note = choice(range(pitches.value[0], pitches.value[1]))
        sequence.append(note + offset)
    return sequence

random_PS_button = widgets.Button(
    description='Make random pitch sequence',
)
output = widgets.Output()

display(random_PS_button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        print(make_random_pitch_sequence(pitches, offset, length))

random_PS_button.on_click(on_button_clicked)

Button(description='Make random pitch sequence', style=ButtonStyle())

Output()

# Notice the repetitions

In [8]:
def get_probs(weights, counts):
    probs = []
    total = 0
    for i in range(len(weights)):
        total += weights[i] * growth_function(counts[i])

    for i in range(len(weights)):
        prob = weights[i] * growth_function(counts[i]) / total
        probs.append(prob)

    return probs

In [9]:
def growth_function(n): # Tenney's convex growth function
    return 2**n

In [10]:
def make_dca_spaced_pitch_sequence(pitches, offset, weights, length, show_data=False):
    counts = [0] * len(pitches)
    pcprobs = get_probs(weights, counts)

    sequence = []
    for note in range(length.value):
        # random choice according to probability
        note_choice = npchoice(pitches, p=pcprobs)
        element = pitches.index(note_choice)
        note = note_choice + offset

        # add note to sequence
        sequence.append(note)

        # now update counts
        for i, pitch in enumerate(pitches):
            if i == element:
                counts[i] = 0
            else:
                counts[i] += 1

        # recalculate pcprobs
        pcprobs = get_probs(weights, counts)

        if show_data:
            print("\nRandom index choice:", note_choice)
            print("Current counts:", counts)
            print("Current probabilities:", pcprobs)

    return sequence

In [11]:
print(make_dca_spaced_pitch_sequence(pitches, offset, weights, length, show_data=True))



Random index choice: 4
Current counts: [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
Current probabilities: [0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.043478260869565216, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043, 0.08695652173913043]

Random index choice: 8
Current counts: [2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2]
Current probabilities: [0.09302325581395349, 0.09302325581395349, 0.09302325581395349, 0.09302325581395349, 0.046511627906976744, 0.09302325581395349, 0.09302325581395349, 0.09302325581395349, 0.023255813953488372, 0.09302325581395349, 0.09302325581395349, 0.09302325581395349]

Random index choice: 3
Current counts: [3, 3, 3, 0, 2, 3, 3, 3, 1, 3, 3, 3]
Current probabilities: [0.10126582278481013, 0.10126582278481013, 0.10126582278481013, 0.012658227848101266, 0.05063291139240506, 0.10126582278481013, 0.10126582278481013, 0.10126582278481013, 0.02531645569620253, 

# After an element is chosen, it's count goes to 0 and it's probability goes to near-zero. Higher counts are biased by the growth function (2**n) to have even higher probabilities.

In [ ]:
length = widgets.IntSlider()
display(length)
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length.value)
print(sequence)

# See any repetitions? Probably not!

# Here's what the pitch frequency looks like:

In [ ]:
for pitch in pitches:
    print(sequence.count(pitch))

# what if we change the weights to favor certain pitches?

In [ ]:
weights = [i * 2 for i in range(1, 13)]

In [ ]:
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
print(sequence)

In [ ]:
for pitch in pitches:
    print(sequence.count(pitch))

# Still some randomness, but definitely skewed towards one side

# Let's reset the weights to an equal value and try that again

In [ ]:
weights = [1] * 12 # how much to weigh each element's probability

# Now let's see what this looks like in notation

In [ ]:
def output_ly(sequence):
    notes = []
    for pitch in sequence:
        duration = abjad.Duration(1, 4)
        note = abjad.Note(pitch, duration)
        notes.append(note)

    staff = abjad.Staff(notes)
    abjad.show(staff)

In [ ]:
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
output_ly(sequence)

# How about changing over time?

1. make weights a function of their distance from a pitch center
2. move that pitch center over time
3. make sequences (clangs) vary in length
4. increase pitch center and sequence average length until 2/3 through form, then decrease

In [ ]:
def make_clang(pitches, weights, length, deviation, offset=0):
    # length = average length +/- deviation range
    length = length + randrange(deviation*-1, deviation)
    sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
    sequence.append("rest") # end every clang with a rest
    return sequence

In [ ]:
# build clangs
total_clangs = 72

# weight by proximity to pitch center
increasing = [ 1 * i for i in range(1, 7) ]
decreasing = list(reversed(increasing))
weights = increasing + decreasing

# make peak 2/3 of the way through form. Tenney does this with a calculation based on the golden mean, but is more/less in the same place
peak = int(total_clangs * (2/3))

length = 5
length_deviation = int(length * 0.5)

offset = -12
pitch_center = 6
clangs = []

In [ ]:
for clang in range(total_clangs):
    pitches = [*range(int(pitch_center) - 6, int(pitch_center+6))]
    #print(pitches)

    sequence = make_clang(pitches, weights, length, length_deviation, offset=offset)
    clangs.append(sequence)

    if clang < peak:
        pitch_center += 0.6
        length += 1
    else:
        pitch_center -= 1.25
        length -= 2
    length_deviation = int(length * 0.5)

In [ ]:
for clang in clangs:
    print(clang)

In [ ]:
def output_clangs(clangs):
    notes = []
    for clang in clangs:
        for pitch in clang:
            if pitch == "rest":
                notes.append(abjad.Rest('r8'))
            else:
                duration = abjad.Duration(1, 8)
                note = abjad.Note(pitch, duration)
                notes.append(note)

    staff = abjad.Staff(notes)
    abjad.show(staff)
    abjad.play(staff)

In [ ]:
output_clangs(clangs)